In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from datetime import date

# Use a service account.
cred = credentials.Certificate('flightsAccount.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

Destinations structure:

In [ ]:
checkDate = 20230524
origin = 'PSR'

doc_ref = db.collection('TestDestinations').document(origin)
doc_ref.set({
    'CheckDate': checkDate,
    'Destinations': ['BGY','TFS']
})

doc_ref = db.collection('TestDestinations').document(origin).collection('Historical').document(f'{checkDate}')
doc_ref.set({
    'CheckDate': checkDate,
    'Origin': origin,
    'Destinations': ['BGY','TFS']
})

In [ ]:
collections = db.collection('TestDestinations').document('VNO').collections()
for collection in collections:
    for doc in collection.stream():
        print(f'{doc.id} => {doc.to_dict()}')

In [ ]:
checkDate = int(str(date.today()).replace('-',''))
# checkDate = 20230524
origin = 'PSR'
ports = ['PSR','VNO','TFS','BGY']


batch = db.batch()

for i in ports:
    ref = db.collection('TestAirpots').document(i)
    batch.set(ref, {
        'Airport': i,
        'AirportName': f'{i}_Port',
        'AirportCity': f'{i}_City',
        'AirportCountry': f'{i}_Country'
    })

batch.commit()

In [9]:
trip = 'VNO-PSR'

doc_ref = db.collection('TestTrips').document(trip)
doc_ref.set({
    'From': trip[:3],
    'To': trip[4:],
    'Active': True,
    'One Way': False,
    'Outbound Date': 20230904,
    'Return Date': 20230924
})

update_time {
  seconds: 1685170194
  nanos: 989850000
}

In [11]:
trip = 'PSR-VNO'

doc_ref = db.collection('TestTrips').document(trip)
doc_ref.set({
    'From': trip[:3],
    'To': trip[4:],
    'Active': True,
    'One Way': False,
    'Outbound Date': 20230703,
    'Return Date': 20230923
})

update_time {
  seconds: 1685170785
  nanos: 759922000
}

In [7]:
def fireTrip(fireDB, trip: str, isActive: bool, outboundDate: int, returnDate: int=0):
    fireDoc = fireDB.collection('Trips').document(trip)
    if returnDate != 0:
        fireDoc.set({
            'From': trip[:3],
            'To': trip[4:],
            'Active': isActive,
            'OutboundDate': outboundDate,
            'OneWay': False,
            'ReturnDate': returnDate
        })
    else:
        fireDoc.set({
            'From': trip[:3],
            'To': trip[4:],
            'Active': isActive,
            'OutboundDate': outboundDate,
            'OneWay': True
        })